> First time use: follow instructions in the README.md file in this directory.



**[PT]** Português

---

**[EN]** English


# Ordens Religiosas


As anotações ao nomes incluem por vezes referências 
a ordens religiosas ou a colégios associados a ordens religiosas

---

# Religious orders

[summary]


In [21]:
from timelinknb import get_db
from ucalumni.config import default_db

db_spec = default_db
db = get_db(db_spec)

### Tabela de expressões usadas para determinar a ordem religiosa nas notas ao nome

---

### Expressions used to determine the religious order

In [22]:
from ucalumni.extractors import ordens

for expression in sorted(ordens.keys()):
    print(f'{expression:35} {ordens.get(expression)}')

3ª                                  Ordem Terceira
agostinho                           Ordem de Santo Agostinho
agostinho descalço                  Ordem de Santo Agostinho (Descalços)
agostinhos descalços                Ordem de Santo Agostinho (Descalços)
alcântara                           Ordem de São Pedro de Alcântara
antónio                             Ordem de Santo António
avis                                Ordem de Avis
beneditino                          Ordem de São Bento
bento                               Ordem de São Bento
bernardo                            Ordem de São Bernardo
carmelita                           Ordem do Carmo
carmo                               Ordem do Carmo
colegial do colégio de jesus        Companhia de Jesus
colégio da graça                    Ordem de Santo Agostinho (Graça)
companhia de jesus                  Companhia de Jesus
cristo                              Ordem de Cristo
descalço de santo agostinho         Ordem de Santo Agostinho (De

#### Ordens religiosas por número de estudantes

---

#### Religious orders by number of students

In [23]:
from timelinknb.pandas import attribute_to_df

rel_orders = attribute_to_df(
                    the_type='ordem-religiosa',
                    person_info=True,
                    more_cols=['uc-entrada','uc-saida','faculdade','titulo','colegio','naturalidade','nome-pai','padre','nome-nota'],
                    sql_echo=False)
rel_orders.drop(['nome-nota.date','nome-nota.obs','padre.date','padre.obs',
                 'ordem-religiosa.date','ordem-religiosa.obs',
                 'padre.date','padre.obs','sex',
                 'naturalidade.date','naturalidade.obs',
                 'nome-pai.date','nome-pai.obs',
                 'faculdade.date','faculdade.obs',
                'titulo.date','titulo.obs',
                'colegio.date','colegio.obs',
                'uc-entrada.date','uc-entrada.obs',
                'uc-saida.date','uc-saida.obs'],axis=1, inplace=True)
rel_orders = rel_orders[rel_orders['uc-entrada'] != '0000-00-00']
rel_orders = rel_orders[~rel_orders.index.duplicated(keep='first')]
print("Number of unique students with religious order:",len(rel_orders.index.unique()))
rel_orders.info()

Number of unique students with religious order: 3033
<class 'pandas.core.frame.DataFrame'>
Index: 3033 entries, 127987 to 319984
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   name             3033 non-null   object
 1   ordem-religiosa  3033 non-null   object
 2   uc-entrada       3033 non-null   object
 3   uc-saida         3033 non-null   object
 4   faculdade        3002 non-null   object
 5   titulo           2926 non-null   object
 6   colegio          124 non-null    object
 7   naturalidade     449 non-null    object
 8   nome-pai         203 non-null    object
 9   padre            1154 non-null   object
 10  nome-nota        3033 non-null   object
dtypes: object(11)
memory usage: 348.9+ KB


#### Amostra

---

#### Sample

In [24]:
rel_orders.head(20).fillna("")

,name,ordem-religiosa,uc-entrada,uc-saida,faculdade,titulo,colegio,naturalidade,nome-pai,padre,nome-nota
id,,,,,,,,,,,
127987,António Cabral,Ordem de São Bernardo,1665-10-15,1666-10-10,Teologia,frei,,,,,frei religioso de São Bernardo
128152,Feliciano Cabral,Ordem de São Bernardo,1699-10-01,1701-10-01,Teologia,frei,,,,,frei religioso da Ordem São Bernardo
128344,Jerónimo Cabral,Ordem de São Bernardo,1756-10-01,1756-10-01,Teologia,frei,,,,,frei monge de São Bernardo
128485,José de Nossa Senhora Cacela,Ordem de São Domingos,1751-10-01,1753-10-01,Teologia,frei,,,,,frei religioso de São Domingos
128659,Bernardo Caceres,Ordem de São Bernardo,1760-10-01,1763-10-01,Teologia,frei,,,,,frei monge de São Bernardo
128662,Francisco de Caceres,Ordem de São Francisco,1566-06-06,1568-01-11,Teologia,frei,,,,,frei da Ordem de São Francisco
128859,Manuel Cabral,Ordem de Santo Agostinho,1588-01-29,1595-12-24,Teologia,frei,,,,sim,padre frei Ordem de Santo Agostinho
129089,Francisco Caetano,Ordem de São Bernardo,1698-10-01,1705-12-18,Teologia,frei,,,,sim,padre frei religioso da Ordem de São Bernardo
129095,Caetano Caldas,Ordem de Santo Agostinho,1770-10-01,1770-10-01,Teologia,frei,,,,,frei eremita de Santo Agostinho


#### Ordens religiosas por número de estudantes

---

#### Religious orders by number of students

In [25]:
rel_ord_count = rel_orders.groupby('ordem-religiosa')['uc-entrada'].agg(['count','min','max'])
rel_ord_count.drop(['frei','padre frei'],axis=0, inplace=True) # correct current bug, fixed for next run
rel_ord_count.sort_values('count',ascending = False, inplace=True)
rel_ord_count[rel_ord_count['count']>10]

,count,min,max
ordem-religiosa,,,
Ordem de São Bernardo,401,1569-00-00,1827-10-29
Ordem de São Bento,374,1548-01-01,1831-06-25
Ordem do Carmo,283,1536-00-00,1829-10-31
Ordem de São Francisco,267,1540-03-09,1816-10-31
Ordem de Santo Agostinho (Graça),250,1560-10-01,1780-05-30
Ordem da Santíssima Trindade,223,1551-10-00,1820-06-19
Ordem de São Domingos,202,1541-10-20,1827-10-30
Ordem de Santo Agostinho,168,1541-11-29,1829-06-03
Ordem de São Jerónimo,162,1550-03-12,1825-10-25


#### Em processo de verificação

Ordens religiosas com menos de 10 estudantes


---


#### Requiring further analysis

Number of religious orders with less than 10 students

In [26]:
print("Number of cases that escape normalization:")
print(rel_ord_count[rel_ord_count['count']<=10][['count']].sum())
print("Sample:")
rel_ord_count[rel_ord_count['count']<=10].head(20)

Number of cases that escape normalization:
count    70
dtype: int64
Sample:


,count,min,max
ordem-religiosa,,,
ordem de s. francisco,7,1562-03-06,1767-10-01
Paulista,5,1752-10-01,1786-10-27
Ordem de Avis,5,1549-10-00,1626-05-13
Ordem de Santo António,4,1601-10-01,1769-10-01
Província Algarves,4,1747-10-01,1761-10-01
"padre,",2,1754-10-01,1768-01-22
Província Algarve,2,1572-10-01,1576-10-01
Ordem S. Franisco,2,1576-10-01,1791-10-22
S. José,2,1692-10-01,1801-10-03


A célula seguinte mostra as variantes de expressões
detectadas para a mesma ordem religiosa.

---

The following cell show the variations of expressions
for a specific order.

In [27]:
order = "Ordem de São Bento"  #  change to check another order

variants = rel_orders[rel_orders['ordem-religiosa'] == order]['nome-nota'].unique()
print(f"Number of expressions for '{order}':",len(variants))
null_ = [print(v) for v in sorted(list(variants))]

Number of expressions for 'Ordem de São Bento': 37
Frei Monge de São Bento
frei da Ordem de São Bento
frei de Bento Lusido
frei de São Bento
frei do colégio de São Bento
frei monge Beneditino
frei monge beneditino
frei monge da Ordem de São Bento do Mosteiro de Sanfins
frei monge de São Bento
frei monge de São Bento do Brasil
frei monge de são Bento
frei padre da Ordem de São Bento
frei padre da Ordem de São Bento do Mosteiro de Pombeiro
frei padre religioso de S. Bento
frei padre religioso de São Bento
frei religioso da Ordem de São Bento
frei religioso de S. Bento
frei religioso de São Bento
frei religioso do colégio de São Bento
frei religioso e S. Bento
frei, monge beneditino
padre e frei da Ordem de São Bento
padre frei da Ordem de São Bento
padre frei de São Bento
padre frei monge de S. Bento
padre frei monge de São Bento
padre frei religioso da Ordem de S. Bento
padre frei religioso da Ordem de São Bento
padre frei religioso de S. Bento
padre frei religioso de São Bento
padre fr

#### Ver tb / See also [023-jesuits.ipynb](023-jesuits.ipynb)